In [ ]:
import os
import sys
import time
import pandas as pd

from linkedin_jobs_scraper import LinkedinScraper, events, query, filters
from resume_matcher.matching import resume_job_desc_match

In [ ]:
# from langchain_ollama import OllamaLLM
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate

load_dotenv()

## LinkedIn Scraping

In [ ]:
JOB_RESULTS = []

def on_data(data):
    JOB_RESULTS.append({
        "title": data.title,
        "company": data.company,
        "location": data.location,
        "description": data.description,
        "link": data.link,
        "date": data.date
    })

def on_error(error):
    print('[ON_ERROR]', error)

def on_end():
    print('[END]')

def scrape_linkedin_jobs(search_term, location = 'Remote', num_jobs = 10):
    global JOB_RESULTS
    scraper = LinkedinScraper(
        chrome_executable_path=None,
        # chrome_driver_path=None,
        headless=True,
        max_workers=1,
        slow_mo=0.5,
        page_load_timeout=20,
        # browser_user_agent=None,
        # proxy=None
    )
    scraper.on(events.Events.DATA, on_data)
    scraper.on(events.Events.ERROR, on_error)
    scraper.on(events.Events.END, on_end)

    queries = [
        query.Query(
            query=search_term,
            options= query.QueryOptions(
                locations = [location],
                apply_link = True,
                limit = num_jobs,
                filters = query.QueryFilters(
                    experience=[filters.ExperienceLevelFilters.ENTRY_LEVEL],
                    type=[filters.TypeFilters.FULL_TIME]
                ),
            ),
        ),
    ]
    
    scraper.run(queries)
    time.sleep(2)
    return JOB_RESULTS.copy()
    

In [ ]:
JOB_QUERY = "AI Engineer"    # Edit as desired
JOB_LOCATION = "Remote"
NUM_JOBS = 20

scraped_jobs = scrape_linkedin_jobs(JOB_QUERY, location=JOB_LOCATION, num_jobs=NUM_JOBS)

print(f"Scraped {len(scraped_jobs)} jobs.")
print(scraped_jobs)  # Display first 5 jobs

## Resume Matching

In [ ]:
# def analyze_resume_vs_jobs(resume_path, job_descriptions):
#     matcher = resume_job_desc_match(resume_path, job_descriptions)
#     job_scores = []
#     for idx, job in enumerate(job_descriptions):
#         score, missing = matcher.match(job['description'])
#         job_scores.append({
#             "job_index": idx,
#             "title": job['title'],
#             "company": job['company'],
#             "score": score,
#             "missing_keywords": missing,
#             "description": job['description'],
#             "link": job['link'],
#         })
#     return job_scores

In [ ]:
# resume_job_desc_match('Manvith_New_Resume.pdf', "We’re super into the work we do and the community we’ve built and think you might be, too.\n\nQ-Centrix is the largest exclusive provider of clinical data management solutions to acute care hospitals. A market disruptor and innovator, Q-Centrix believes that there is nothing more valuable than clinical data as it is critical in delivering safe, consistent, quality healthcare for all. Bringing together deep expertise across providers, clinical knowledge, data and software, Q-Centrix provides an integrated approach that can redefine and streamline the data management and real-world application process for the healthcare industry, thereby enabling increased efficiency and exciting new solution opportunities.\n\nProviding the industry’s first Enterprise Clinical Data Management (eCDM™) platform, Q-Centrix utilizes its market-leading software, the largest and broadest team of clinical data experts, a modern-stack software and reporting data structure, and best practices from its 1,200+ hospital partners to securely extract, curate, structure, and enhance clinical data at the highest quality level. The resulting high quality structured clinical data is then utilized to support reporting demands, drive improved care delivery, meet financial and operational needs, enable population health workflows and power broad research use cases. Its solutions cover a breadth of clinical segments, including cardiovascular, oncology, infection prevention, trauma and real-world data applications. Q-Centrix’s platform enables its partners to access valuable clinical information that may otherwise be trapped across multiple workflow systems and clinical information platforms. Q-Centrix is positioned for continued growth as they integrate new capabilities and business lines.\n\nJob Summary\n\nThis role focuses on deploying AI-powered solutions to extract structured data from unstructured medical charts. The AI Engineer will work with technical and clinical teams to bring AI models into real-world use, improving efficiency in healthcare data processing while ensuring accuracy, usability and responsible AI practices.\n\nEssential Functions\n\n\nDevelop, test, and deploy LLM-powered extraction pipelines for clinical text at scale.\nAutomate prompt execution, result validation, and error handling to enhance reliability.\nMonitor and maintain production AI models, ensuring uptime, accuracy, and compliance.\nWork with product, data, engineering, and clinical teams to integrate AI outputs into real-world solutions.\nEvaluate and integrate third-party AI solutions when appropriate.\nStay current with advancements in LLM deployment, model serving, and responsible AI practices.\n\n\nRequired Skills/Abilities\n\n\n1+ years of experience in LLM evaluation\nStrong software engineering foundation with proficiency in Python.\nFamiliarity with cloud platforms, data engineering and pipeline automation.\nBachelor’s degree in Computer Science, AI, a related field, or equivalent work experience.\nStrong cross-functional collaboration skills.\nProven track record of thriving in a start-up environment.\n\n\nPreferred Education And Experience\n\n\n2+ years of experience in AI deployment or applied Machine Learning.\nAdvanced degree in Computer Science or related field.\nExperience integrating AI with healthcare data (EHRs, clinical databases).\nExperience working in regulated industries with an understanding of security, compliance, and responsible AI.\n\n\nSupervisory Responsibilities: None\n\nWork environment/Physical Demands: Continuous sitting and fine manipulation.\n\nTravel Requirements: None\n\nWork Authorization: Legally able to work in the United States without sponsorship\n\nTotal Rewards\n\nAt Q-Centrix, our purpose—safer, consistent, quality healthcare for all—drives everything we do. To accomplish this important work, we aim to attract, engage, and retain a talented team by offering a compelling, equitable rewards package. This includes an inclusive culture, a flexible work environment, learning and development opportunities, and robust benefits that support both health and financial wellness. In addition, our supportive community fosters collaboration, learning, growth, and enjoyment, making Q-Centrix a place where meaningful work and a positive work experience go hand in hand. It’s no wonder we’ve earned the Great Place to Work distinction multiple years in a row! The target salary range for this role is $100,000.00 to $120,000.00 per year. An individual’s salary within this range is based on multiple factors including but not limited to skills, experiences, licensure, certifications, and other business and organizational considerations.\n\nIn addition to our inclusive and innovative working environment and comprehensive compensation package, team members enjoy:\n\n\nRemote/hybrid flexibility (depending on location) and a generous time off program with additional paid time for volunteering.\nRobust benefits package including medical, vision, dental, health savings accounts, company paid short- and long-term disability, employee assistance program, paid parental leave, life insurance, accident insurance, and other voluntary benefit programs for employees and their eligible dependents.\n401(k) retirement plan with a company match.\nOpportunities for professional development.\n\n\nCommitment To Diversity, Equity, Inclusion And Belonging\n\nAt Q-Centrix, we hire people who love learning, value innovation, and believe in our purpose of safer, consistent, quality health care for all. We applaud qualified applicants who are accountable and committed to producing quality work. As an Equal Opportunity Employer, we support and value diversity, dignity, and respect in our work environment, and are committed to creating an inclusive environment in which everyone can thrive.\n\nWe employ people based on the needs of the business and the job, and their individual professional qualifications. Here’s what does not impact our employment decisions: race, religious creed, religion, color, sex, sexual orientation, pregnancy, parental status, genetic information, gender, gender identity, gender expression, age, national origin, ancestry, citizenship, protected veteran or disability status, health, marital, civil union or domestic partnership status, or any status or characteristic protected by the laws or regulations in locations where we operate. If you are an individual with a qualified disability and you need an accommodation during the interview process, please reach out to your recruiter.\n\nCandidate Privacy Statements\nShow more")

In [ ]:
import docx2txt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def resume_job_desc_match_2(resume,job_desc):
    # job_desc = docx2txt.process(job_desc)
    # resume = docx2txt.process(resume)
    content = [resume,job_desc]
    cv = CountVectorizer()
    matrix = cv.fit_transform(content)
    similarity_matrix = cosine_similarity(matrix)
    match_percentage = similarity_matrix[0][1]*100
    return f"{round(match_percentage, 2)}%"

In [ ]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text()
    return text

# Usage
pdf_text = extract_text_from_pdf("C:/Users/10839330/OneDrive - LTIMindtree/Desktop/Ollama_Agents/Manvith_New_Resume.pdf")
print(pdf_text)



In [ ]:
resume_job_desc_match_2(pdf_text, scraped_jobs[0]['description'])

In [ ]:
scored_jobs = []
for i, job in enumerate(scraped_jobs):
    print(f"Job {i+1}: {job['title']} at {job['company']}")
    print(f"Location: {job['location']}")
    print(f"Description: {job['description'][:100]}...")  # Print first 100 characters
    print(f"Link: {job['link']}")
    print(f"Date Posted: {job['date']}\n")
    scored_jobs.append({
        "job_index": i,
        "title": job['title'],
        "company": job['company'],
        "score": resume_job_desc_match_2(pdf_text, job['description']),
        "description": job['description'],
        "link": job['link'],
    })
    print(resume_job_desc_match_2(pdf_text, scraped_jobs[i]['description']))
    print("\n\n\n")

## Building Resume Enhancing agent

In [ ]:
def get_ollama_suggestions(resume_text, job_description):
    prompt = (
        "You are a career coach and resume expert. Given the following resume and job description, "
        "suggest specific improvements to the resume to fit this particular job's requirements. "
        "Resume:\n%s\n\nJob Description:\n%s\n\n"
        "Return bullet points with concrete suggestions only, and add the missing keywords required."
    ) % (resume_text, job_description)
    # llm = OllamaLLM(
    #     model = model,
    #     temperature=0.2,
    # )
    llm = ChatNVIDIA(
        model = os.getenv("NVIDIA_MODEL_NAME"),
        api_key = os.getenv("NVIDIA_API_KEY"),
    )

    response = llm.invoke(prompt)
    return response

In [ ]:
scored_jobs = sorted(scored_jobs, key= lambda x: x['score'], reverse= True)[:5]

In [ ]:
print(scored_jobs)

In [ ]:
final_results = []

for job in scored_jobs:
    suggestions = get_ollama_suggestions(resume_text= pdf_text, job_description= job['description'])
    final_results.append({
            'title': job['title'],
            'company': job['company'],
            'score': job['score'],
            'link': job['link'],
            'suggestions': suggestions
        })

print("\n--- Top Matching Jobs and Personalized Suggestions ---\n")
for job in final_results:
    print(f"[{job['title']} at {job['company']}] (Score: {job['score']}%)")
    print(f"Link: {job['link']}")
    print("Improvement Suggestions:\n", job['suggestions'])
    print("-" * 60)